# Assignmnet 1_LI XINYA

In [1]:
# import the libraries
import pandas as pd
import numpy as np
from numpy.random import randn
import torch
from torch import nn
import warnings
warnings.filterwarnings('ignore')

## Q1. Implement the neural network in pytorch

1. Input (x1,x2): 2 nodes
2. First hidden layer: 10 nodes, with weights (w) and bias (b), sigmoid activation function
3. Second hidden layer: 10 nodes, with weights (w) and bias (b), sigmoid activation function
4. Output (predict): 1 node

In [2]:
input_shape = 2
output_shape = 1

#define the neural network model
network_model = torch.nn.Sequential(
    # define input layer
    torch.nn.Linear(input_shape, 10),
    nn.Sigmoid(),
    # define first hidden layer
    torch.nn.Linear(10, 10),
    nn.Sigmoid(),
    # define second hidden layer
    torch.nn.Linear(10, output_shape)
)

print(network_model)

Sequential(
  (0): Linear(in_features=2, out_features=10, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=10, out_features=1, bias=True)
)


## Q2. Generate the input date
(x1,x2) \in [0,1] drawn from a uniform random distribution

In [3]:
x1 = np.random.uniform(0,1)
x2 = np.random.uniform(0,1)
x_input = torch.tensor([x1,x2])
x = np.array([round(x1,4),round(x2,4)])
print(x,x_input)

[0.9274 0.6714] tensor([0.9274, 0.6714])


## Q3. Generate the labels y

In [4]:
y_label = torch.tensor((x1*x1+x2*x2)/2)
y = np.array([(x1*x1+x2*x2)/2])

## Q4. Implement a loss function L

In [5]:
def loss_function(y_pred, y_label):
    return torch.sum((y_pred-y_label)**2)

## Q5. Forward propagation

In [6]:
optimizer = torch.optim.SGD(network_model.parameters(), lr=0.1)

optimizer.zero_grad()
y_pred = network_model(x_input)
loss = loss_function(y_pred, y_label)
loss.backward()

print("Loss = " + str(loss.item()))

Loss = 0.43650704622268677


## Q6. Compute the gradients using pytorch autograd

In [7]:
input_layer = network_model[0]
hidden_layer1 = network_model[2]
hidden_layer2 = network_model[4]

In [8]:
print("Loss = " + str(np.round(loss.item(),4)))
print("Input layer weight gradient: " + str(np.round(input_layer.weight.grad.tolist(),4)))
print("Input layer bias gradient: " + str(np.round(input_layer.bias.grad.tolist(),4)))
print("Hidden layer 1 weight gradient: " + str(np.round(hidden_layer1.weight.grad.tolist(),4)))
print("Hidden layer 1 bias gradient: " + str(np.round(hidden_layer1.bias.grad.tolist(),4)))
print("Hidden layer 2 weight gradient: " + str(np.round(hidden_layer2.weight.grad.tolist()[0],4)))
print("Hidden layer 2 bias gradient: " + str(np.round(hidden_layer2.bias.grad.tolist(),4)))
print("y prediction:" + str(np.round(y_pred.tolist(),4)))

Loss = 0.4365
Input layer weight gradient: [[-0.0116 -0.0084]
 [-0.0023 -0.0016]
 [ 0.0071  0.0052]
 [ 0.0058  0.0042]
 [ 0.0057  0.0041]
 [-0.0028 -0.002 ]
 [-0.0103 -0.0074]
 [ 0.0069  0.005 ]
 [-0.0007 -0.0005]
 [-0.0047 -0.0034]]
Input layer bias gradient: [-0.0125 -0.0024  0.0077  0.0063  0.0061 -0.003  -0.0111  0.0074 -0.0007
 -0.0051]
Hidden layer 1 weight gradient: [[-0.0298 -0.0415 -0.0319 -0.0141 -0.0428 -0.0245 -0.0404 -0.0172 -0.0253
  -0.0284]
 [ 0.0231  0.0321  0.0246  0.0109  0.0331  0.0189  0.0312  0.0133  0.0196
   0.022 ]
 [ 0.0455  0.0634  0.0486  0.0215  0.0654  0.0373  0.0617  0.0262  0.0386
   0.0434]
 [-0.0192 -0.0268 -0.0205 -0.0091 -0.0276 -0.0158 -0.026  -0.0111 -0.0163
  -0.0183]
 [-0.035  -0.0487 -0.0374 -0.0165 -0.0502 -0.0287 -0.0474 -0.0202 -0.0296
  -0.0333]
 [ 0.0205  0.0286  0.0219  0.0097  0.0295  0.0168  0.0278  0.0118  0.0174
   0.0195]
 [-0.0215 -0.03   -0.023  -0.0102 -0.0309 -0.0176 -0.0291 -0.0124 -0.0182
  -0.0205]
 [ 0.0147  0.0204  0.0157  0.

## Q7  Implement the forward propagation and backpropagation algorithm from scratch

In [9]:
# Define weight and bias
w1 = input_layer.weight.t().detach().numpy()
w2 = hidden_layer1.weight.t().detach().numpy()
w3 = hidden_layer2.weight.t().detach().numpy()
b1 = input_layer.bias.detach().numpy()
b2 = hidden_layer1.bias.detach().numpy()
b3 = hidden_layer2.bias.detach().numpy()

In [10]:
for t in range(1):
    # Forward pass
    h1 = x.dot(w1)+b1
    h1_ = 1.0 / (1.0 + np.exp(-h1))
    h2 = h1_.dot(w2)+b2
    h2_ = 1.0 / (1.0 + np.exp(-h2))
    y_pred_ = h2_.dot(w3)+b3
    loss_ = np.square(y_pred_ - y).sum()
    
    # Backward pass
    dy_pred = 2.0 * (y_pred_ - y)
    
    dt = float(dy_pred)
    dw3 = np.dot(h2_.T, dt)
    db3 = np.ones(1).dot(dt)
    
    dt = np.dot(dt, w3.T)*h2_*(1-h2_)
    dw2 = np.dot(h1_.reshape(len(h1_),1), dt)
    db2 = np.ones(1).dot(dt)
    
    dt = np.dot(dt, w2.T)*h1_*(1-h1_)
    dw1 = np.dot(x.reshape(len(x),1), dt)
    db1 = np.ones(1).dot(dt)
    
    w1 -= 0.1 * dw1 # learning rate is 0.1
    w2 -= 0.1 * dw2
    w3 -= 0.1 * dw3.reshape(len(dw3),1)
    
    b1 -= 0.1 * db1 
    b2 -= 0.1 * db2
    b3 -= 0.1 * db3
    
print("Loss = " + str(round(loss_,4)))    
print("Input layer weight gradient: " + str(np.round(dw1.T,4)))
print("Input layer bias gradient: " +  str(np.round(db1,4)))
print("Hidden layer 1 weight gradient: " + str(np.round(dw2,4)))
print("Hidden layer 1 bias gradient: " + str(np.round(db2,4)))
print("Hidden layer 2 weight gradient: " + str(np.round(dw3,4)))   
print("Hidden layer 2 bias gradient: " + str(np.round(db3,4)))    
print("y prediction: " + str(np.round(y_pred_,4)))

Loss = 0.4365
Input layer weight gradient: [[-0.0116 -0.0084]
 [-0.0023 -0.0016]
 [ 0.0071  0.0052]
 [ 0.0058  0.0042]
 [ 0.0057  0.0041]
 [-0.0028 -0.002 ]
 [-0.0103 -0.0074]
 [ 0.0069  0.005 ]
 [-0.0007 -0.0005]
 [-0.0047 -0.0034]]
Input layer bias gradient: [-0.0125 -0.0024  0.0077  0.0063  0.0061 -0.003  -0.0111  0.0074 -0.0007
 -0.0051]
Hidden layer 1 weight gradient: [[-0.0298  0.0231  0.0455 -0.0192 -0.035   0.0205 -0.0215  0.0147 -0.0367
  -0.0248]
 [-0.0415  0.0321  0.0634 -0.0268 -0.0487  0.0286 -0.03    0.0204 -0.0511
  -0.0345]
 [-0.0319  0.0246  0.0486 -0.0205 -0.0374  0.0219 -0.023   0.0157 -0.0392
  -0.0265]
 [-0.0141  0.0109  0.0215 -0.0091 -0.0165  0.0097 -0.0102  0.0069 -0.0173
  -0.0117]
 [-0.0428  0.0331  0.0654 -0.0276 -0.0502  0.0295 -0.0309  0.0211 -0.0527
  -0.0355]
 [-0.0245  0.0189  0.0374 -0.0158 -0.0287  0.0168 -0.0176  0.012  -0.0301
  -0.0203]
 [-0.0404  0.0312  0.0617 -0.026  -0.0474  0.0278 -0.0291  0.0199 -0.0497
  -0.0335]
 [-0.0172  0.0133  0.0262 -0.

## Q8. Numerical precision errors

In [11]:
from sklearn.metrics import mean_squared_error

In [12]:
a = np.round(loss.item(),4)
A = round(loss_,4)

b = np.round(input_layer.weight.grad.tolist(),4)
B = np.round(dw1.T,4)

c = np.round(input_layer.bias.grad.tolist(),4)
C = np.round(db1,4)

d = np.round(hidden_layer1.weight.grad.tolist(),4)
D = np.round(dw2,4)

e = np.round(hidden_layer1.bias.grad.tolist(),4)
E = np.round(db2,4)

f = np.round(hidden_layer2.weight.grad.tolist()[0],4)
F = np.round(dw3,4)

g = np.round(hidden_layer2.bias.grad.tolist(),4)
G = np.round(db3,4)

h = np.round(y_pred.detach().numpy())
H = np.round(y_pred_)

In [13]:
print(f,F)

[-0.6824 -0.5496 -0.709  -0.6587 -0.6588 -0.7589 -0.5797 -0.6423 -0.6978
 -0.7537] [-0.6824 -0.5496 -0.709  -0.6587 -0.6588 -0.7589 -0.5797 -0.6423 -0.6978
 -0.7537]


In [14]:
print(a-A)
print(mean_squared_error(b,B))
print(mean_squared_error(c,C))
print(mean_squared_error(d,D))
print(mean_squared_error(e,E))
print(mean_squared_error(f,F))
print(mean_squared_error(g,G))
print((h-H)[0])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
